<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/tda2023_xlmr_stransformer_2019data_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import transformers
import datasets

In [3]:
import datetime
dset=datasets.load_dataset("json",
                           data_files={"en":"http://dl.turkunlp.org/TKO_8964_2023/news-en-2019.jsonl", 
                                       "fi":"http://dl.turkunlp.org/TKO_8964_2023/news-fi-2019.jsonl"})

## Some simple sorting and filtering operations can be done directly on the dataset
dset=dset.sort("timestamp")
# dset=dset.filter(lambda example: example["timestamp"]>datetime.datetime(2019,12,1,0,0))
print(dset)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating en split: 0 examples [00:00, ? examples/s]

Generating fi split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-00f060784ba4a285/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    en: Dataset({
        features: ['summary', 'tags', 'text', 'timestamp', 'title', 'url'],
        num_rows: 2481
    })
    fi: Dataset({
        features: ['summary', 'tags', 'text', 'timestamp', 'title', 'url'],
        num_rows: 24015
    })
})


In [4]:
for elem in dset["en"].select(range(10)):
    print(elem["timestamp"],elem["title"])



2018-12-26 10:22:56 Holiday return traffic peaking Wednesday
2018-12-26 12:30:00 Watch: Reindeer alert app makes Lapland roads safer for drivers
2018-12-26 15:00:00 Finland loosens name law, but most still prefer classics
2018-12-27 04:45:10 Thursday's papers: Fireworks ban, ski lift sabotage and most desirable employers
2018-12-27 10:12:56 Thousands of overseas applicants vie for seasonal jobs in Lapland
2018-12-27 11:30:00 Hike benefits to tackle lengthening food queues, academic warns
2018-12-27 13:29:39 THL: Microbes pose no threat to winter swimmers
2018-12-27 15:00:00 Suspect in Helsinki child murder had history of violent crime, records show
2018-12-27 15:45:00 Finland's Border Guards brace for seasonal influx of Russian tourists
2018-12-27 16:45:00 Finland's most spectacular gingerbread house is from Kuopio


# Computing transformer embeddings

* The easiest way is to use the `pipeline` abstraction
* You give it a model, a task, and a hint on where to run the model (CPU vs GPU)
* The task of extracting the embeddings is called `feature-extraction`
* truncation: dealing with too long inputs
* batch_size: batching for efficiency

In [5]:
import transformers

## This loads the basic XLM-R model
#p=transformers.pipeline(task="feature-extraction",model="xlm-roberta-base",return_tensors=True,device=0)

## This loads the appropriate SentenceTransformer model
p=transformers.pipeline(task="feature-extraction",model="sentence-transformers/paraphrase-xlm-r-multilingual-v1", tokenizer="sentence-transformers/paraphrase-xlm-r-multilingual-v1",return_tensors=True,device=0)

## This call I copied from here:
## https://huggingface.co/docs/transformers/main_classes/pipelines#pipeline-batching
fi_embedded=p(transformers.pipelines.pt_utils.KeyDataset(dset["fi"], "title"), batch_size=64, truncation="only_first")
en_embedded=p(transformers.pipelines.pt_utils.KeyDataset(dset["en"], "title"), batch_size=64, truncation="only_first")

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

## Runtime efficiency and device placement

* The pipeline processes examples in batches and yields the results
* The results are by default placed on the same device as the model, in our case the GPU
* We do don't want to have these stick around for too long
* Below is an attempt at efficient processing:
  1. Pool the token embeddings right away by averaging
  2. Move the result out from the GPU right away
* We end up with all data embedded, and nicely in a list

In [6]:
import torch 
fi_embedded_pooled=[torch.mean(elem,axis=1) for elem in fi_embedded]
en_embedded_pooled=[torch.mean(elem,axis=1) for elem in en_embedded]

* Every result is a 768-long embedding of one input document
* We can cash in a doc_count-by-768 tensor by stacking these in torch

In [7]:
print(fi_embedded_pooled[0].shape)
print(len(fi_embedded_pooled))

torch.Size([1, 768])
24015


In [8]:
all_fi=torch.vstack(fi_embedded_pooled)
all_en=torch.vstack(en_embedded_pooled)

# Cosine similarity

* We can now compare all embeddings together
* There is a handy tool for that in sklearn
* But for that to work, we need to turn the torch tensors into numpy ndarrays (`.numpy()` method)

In [9]:
import sklearn.metrics
M=sklearn.metrics.pairwise.cosine_similarity(all_en.numpy(),all_fi.numpy())



* To get the id of the most similar document to any other document, argsort is just the right tool

In [10]:
import numpy
print(M.shape)
aligned=numpy.argsort(-M,axis=-1) # -M so that we get a descending sort

(2481, 24015)


*   We need to manage our expectations here
*   Not every piece of news will have another one corresponding to it
* Definitely not if we compare the English and Finnish data, since there is so much less in the English data
* So we should look at pairs in their descending order of similarity
* ...to get the best matches first
* There is a (somewhat mind-bending) way to achieve this in NumPy, but we can go about this with a simple loop here
* when comparing within one language (i.e. you run all embeddings against each other), the first hit is naturally the example itself



In [11]:
pairs=[]
for i in range(M.shape[0]): #M.shape[0] is the number of rows / input documents
    j=aligned[i,0] # 1 is the 2nd best hit, ie the 1st best which is not the document itself
    score=M[i,j]
    pairs.append((i,j,score))
pairs.sort(key=lambda ijs:ijs[2],reverse=True) #Sort the matches in descending order of their similarity

In [12]:
for i,j,score in pairs[:30]:
    print(score)
    print(dset["en"]["title"][i])
    print(dset["fi"]["title"][j])
    print()

0.9419267
Presidents Niinistö and Macron meet in Paris
Presidentti Niinistö ja Macron tapaavat tänään Pariisissa

0.92731106
Survey: Only six percent of young people believe climate change will be controlled
Kysely: Vain kuusi prosenttia nuorista uskoo, että ilmastonmuutoksen hillinnässä onnistutaan

0.9134449
Foreign Minister Haavisto to negotiate on Sudan for EU
Ulkoministeri Pekka Haavisto neuvottelemaan Sudanin tilanteesta EU:n puolesta

0.9126774
Stora Enso cutting up to 400 jobs in Oulu
Stora Enso aloittaa yt-neuvottelut Oulussa – paperikone suljetaan, 400 työpaikkaa uhattuna

0.89801073
Supo: Large numbers of foreign intelligence staff in Finland; Russia and China particularly active
Supo: Suomessa on suuri määrä ulkomaisten tiedustelupalveluiden henkilöstöä – erityisesti Venäjä ja Kiina aktiivisia

0.8855576
35,000 more employed than one year ago
Tilastokeskus: Työllisiä 35 000 enemmän kuin vuosi sitten

0.88420236
Putin to visit Helsinki next Wednesday
Putin Suomeen vierailull